In [40]:
import numpy as np

In [41]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("~/Downloads/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin", binary=True, unicode_errors="ignore")

In [42]:
import pandas as pd

In [219]:
dico = pd.read_csv("/home/olojkine/Downloads/lexique-grammalecte-fr-v7.0.txt", skiprows=15, sep='\t')
dico.sort_values("Fréquence", inplace=True)
dico = dico[(dico.Fréquence > 1e-5) & (dico.Étiquettes.str.match("nom|v[123]|adj")) & (dico.Lemme.str.len() > 2)]
dico = dico[dico.Lemme.isin(model.index_to_key)]
dico.drop_duplicates("Lemme", inplace=True)
dico[["Lemme", "Fréquence"]]

,Lemme,Fréquence
91475,dopaminergique,0.000010
91473,voleter,0.000010
91472,tabler,0.000010
91469,isométrique,0.000010
91468,combiner,0.000010
...,...,...
82,temps,0.105090
71,autre,0.134264
42,pas,0.275864
38,plus,0.281590


In [220]:
import requests
import time
def req_word(word):
    r = requests.post("https://cemantix.herokuapp.com/score", data={"word": word}).json()
    if "error" in r and "tapez trop vite" in r["error"]:
        time.sleep(.2)
        return req_word(word)
    return r.get("score", np.nan)

In [229]:
import time
t = time.time()
vocab = [(x.Lemme, x.Fréquence) for _,x in dico.iterrows()]
i = 0
while vocab:
    i += 1
    word = max(vocab, key=lambda tup: tup[1])[0]
    print(f"{i:3d}. J'essaie “{word}”. ", end="")
    score = req_word(word)
    if np.isnan(score):
        print(" Cémantix ne connaît pas ce mot.")
        vocab = [(w, prev) for w, prev in vocab if w != word]
        continue
    if score == 1:
        print(f" GAGNÉ en {time.time() - t:.1f} secondes")
        break
    print(f"Score: {100*score:.2f}.")
    dmin = 0.15
    sims = {
        word: sim
        for word, sim
        in model.most_similar(word, topn=450_000)
    }
    vocab = [(w, prev / (1e-5 + abs(score-sims.get(w, 1)))) for w, prev in vocab if w != word]

  1. J'essaie “est”. Score: -1.48.
  2. J'essaie “paiement”. Score: 3.85.
  3. J'essaie “fleuve”. Score: -3.76.
  4. J'essaie “logement”. Score: -0.05.
  5. J'essaie “micro”.  GAGNÉ en 2.7 secondes
